In [107]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [108]:
# sample list
mapping_16S = pd.read_csv("~/Documents/thesis/16S/qiime2/mappingfiles/16S_mapping.csv")
mapping_16S.head()

,#SampleID,Mother_Child,SubjectID,MaternalID,TimePoint,Mgx_batch Mapping,16S_batch Mapping
0,C0052-5F-1A,C,52,0052_m,5,Mgx_batch001,16S_batch001
1,C0126-8F-1A,C,126,0126_m,8,Mgx_batch001,16S_batch001
2,C0126-8F-1B,C,126,0126_m,8,Mgx_batch001,16S_batch001
3,C0127-6F-1B,C,127,0127_m,6,Mgx_batch001,16S_batch001
4,C0286-5F-1A,C,286,0286_m,5,Mgx_batch001,16S_batch001


In [109]:
samplenames = list(mapping_16S["#SampleID"])

In [78]:
len(samplenames)

345

In [143]:
# read in mgx data
mgx = pd.read_csv("mgx_genera.csv")
mgx.rename(columns={'Taxon':'taxa'}, inplace=True)
mgx.head()


,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
0,Acidaminococcaceae_unclassified,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
1,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.002632,0.0,0.0,0.000000,0.03501,0.0,0.0
2,Acinetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0
3,Actinobacillus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000139,0.000000,0.0,0.0,0.000116,0.00000,0.0,0.0
4,Actinobaculum,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00000,0.0,0.0


In [144]:
# ratio of unclassified organisms profiled with mgx
sum(mgx.taxa.str.contains("_unclassified| noname"))/mgx.shape[0]

0.041916167664670656

In [5]:
# remove taxa that are unclassified or have no name
# "_unclassified"
# "_noname"
mgx = mgx[~mgx.taxa.str.contains("_unclassified")]
mgx = mgx[~mgx.taxa.str.contains("_noname")]

In [6]:
mgx.sample(15)

,taxa,C0052-5F-1A,C0126-8F-1A,C0126-8F-1B,C0127-6F-1B,C0286-5F-1A,C0296-5F-1A,C0053-6F-1A,C0108-4F-1A,C0498-1F-2A,...,M1116-1F-1A,C0828-4F-1A,C1062-3F-1A,M1082-1F-1A,C0461-4F-1A,M1110-1F-1A,C0388-7F-1A,M1088-1F-1A,C1089-1F-1A,M1109-1F-1A
61,Eikenella,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
114,Parabacteroides,0.018743,0.004718,0.005318,0.012280,0.053458,0.017058,0.028365,0.034533,0.000000,...,0.044239,0.000000,0.032096,0.032052,0.000000,0.023293,0.025875,0.041084,0.026649,0.010695
153,Streptococcus,0.005149,0.002839,0.002457,0.003313,0.004065,0.011671,0.038848,0.011740,0.002175,...,0.007436,0.048392,0.002135,0.004748,0.004127,0.000598,0.007043,0.001420,0.001958,0.002092
48,Coprobacillus,0.000000,0.000000,0.000000,0.000000,0.002431,0.000000,0.000000,0.000000,0.000081,...,0.000000,0.000000,0.011109,0.000000,0.000947,0.000000,0.000000,0.000000,0.000000,0.000000
141,Saccharomyces,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,Anaerofustis,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
37,Cetobacterium,0.000000,0.000000,0.000000,0.000000,0.000446,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
160,Tunalikevirus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
89,Lactococcus,0.000504,0.000000,0.000000,0.000000,0.000000,0.000222,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000160,0.000000,0.000000,0.000000,0.000093,0.000000
15,Anaeroglobus,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [7]:
mgx.to_csv('mgx_abundance.csv', index=False)

In [135]:
# read in 16S data
amplicon = pd.read_csv("feature-table.csv")
amplicon.rename(columns={'#OTU ID':'taxa'}, inplace=True)


In [136]:
# clean taxonomic names
# keep only genera
amplicon["taxa"] = amplicon['taxa'].str.split("D_5__", expand=True)[1]
amplicon.dropna(inplace=True)


In [137]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
0,Methanobrevibacter,0,24,0,0,0,0,0,0,0,...,0,0,0,0,0,0,19,0,0,0
7,Subgroup 10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Subgroup 23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Ilumatobacter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,Actinomyces,0,0,7,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
len(amplicon.columns.values)

264

In [139]:
amplicon.sort_values(by = "taxa", ascending=True, inplace=True)

In [140]:
amplicon["taxa"] = amplicon["taxa"].astype(str)

In [142]:
# ratio of unclassified genera for 16S profiling

sum(amplicon.taxa.str.contains("_unclassified| unidentified| uncultured| group"))/amplicon.shape[0]

0.06435643564356436

In [13]:
# remove any Nones, uncultured organisms
amplicon = amplicon[~amplicon.taxa.str.contains("uncultured")]
amplicon = amplicon[~amplicon.taxa.str.contains("metagenome")]
amplicon = amplicon[~amplicon.taxa.str.contains("unidentified")]
amplicon = amplicon[~amplicon.taxa.str.contains(r'[0-9]')]
amplicon = amplicon[~amplicon.taxa.str.contains("group")]

In [14]:
# calculate relative abundances
bug_sum = amplicon.sum(axis = 0, skipna = True)
amplicon.iloc[:, 1:] = amplicon.iloc[:, 1:]/bug_sum

In [15]:
amplicon.to_csv('16S_abundance.csv')

### looking for the intersection of taxonomy

In [16]:
# calculate number of taxa in both vs. either one

In [17]:
amplicon_taxa_list = set(amplicon["taxa"])
mgx_taxa_list = set(mgx["taxa"])

In [18]:
intersection = [bug for bug in amplicon_taxa_list if bug in mgx_taxa_list]

def union(lst1, lst2): 
    final_list = list(set(lst1) | set(lst2)) 
    return final_list 

union = union(list(set(amplicon_taxa_list)), list(set(mgx_taxa_list)))


In [19]:
union_df = pd.DataFrame(union)


In [20]:
union_df.head()

,0
0,Lambdalikevirus
1,Allorhizobium-Neorhizobium-Pararhizobium-Rhizo...
2,Formosa
3,Pediococcus
4,Gluconobacter


In [21]:
union_df.to_csv('taxalist.csv', index=False)

len(intersection)
len(amplicon_taxa_list)-len(intersection)
len(mgx_taxa_list)-len(intersection)

In [22]:
amplicon_only = [bug for bug in amplicon_taxa_list if bug not in intersection]
mgx_only = [bug for bug in mgx_taxa_list if bug not in intersection]


In [145]:
amplicon_only

['Allorhizobium-Neorhizobium-Pararhizobium-Rhizobium',
 'Formosa',
 'Terrisporobacter',
 'Acholeplasma',
 'Agathobacter',
 'Rikenella',
 'Howardella',
 'Flavobacterium',
 'Lautropia',
 'Faecalitalea',
 'Fusicatenibacter',
 'Eudoraea',
 'Sphaerochaeta',
 'Winogradskyella',
 'Geobacter',
 'Ilumatobacter',
 'Rubritalea',
 'Alteromonas',
 'Anaerovibrio',
 'Rubricoccus',
 'Desulfosarcina',
 'Romboutsia',
 'Vibrio',
 'Anaeroplasma',
 'Sanguibacteroides',
 'Medicago truncatula (barrel medic)',
 'Lentimicrobium',
 'Desulfobacter',
 'Flavitalea',
 'Flavisolibacter',
 'Nitrospira',
 'Candidatus Soleaferrea',
 'Granulosicoccus',
 'Clade Ia',
 'Rhodoferax',
 'Roseibacillus',
 'Amylibacter',
 'Loktanella',
 'Lutibacter',
 'Woeseia',
 'Noviherbaspirillum',
 'Shuttleworthia',
 'Dielma',
 'Blastopirellula',
 'Anaerosporobacter',
 'Desulfocapsa',
 'Intestinibacter',
 'Luteolibacter',
 'Psychroserpens',
 'Duganella',
 'Lysinibacillus',
 'Algibacter',
 'Epulopiscium',
 'Litorimonas',
 'Lutimonas',
 'Acti

### creating long-form dataframe with data from both sequencing methods

In [23]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,M1087-1F-1A,M1088-1F-1A,M1095-1F-1A,M1096-1F-1A,M1099-1F-1A,M1109-1F-1A,M1110-1F-1A,M1113-1F-1A,M1115-1F-1A,M1116-1F-1A
245,Acetanaerobacterium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,Acetobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
438,Acholeplasma,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
387,Achromobacter,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
308,Acidaminococcus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.011094,0.009139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
mgx_unique = [sample for sample in list(mgx.columns.values) if sample not in list(amplicon.columns.values)] 
amplicon_unique = [sample for sample in list(amplicon.columns.values) if sample not in list(mgx.columns.values)]


mgx.drop(mgx_unique, axis=1, inplace=True)
amplicon.drop(amplicon_unique, axis=1, inplace=True)

In [25]:
len(mgx.columns.values)

262

In [26]:
len(amplicon.columns.values)

262

In [27]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [28]:
mgx_melt = pd.melt(mgx, id_vars=["taxa"], var_name = "sampleid", value_name = "mgx_abund")

In [29]:
mgx_melt.head()

,taxa,sampleid,mgx_abund
0,Acidaminococcus,M0652-1F-1A,0.0
1,Acinetobacter,M0652-1F-1A,0.0
2,Actinobacillus,M0652-1F-1A,0.0
3,Actinobaculum,M0652-1F-1A,0.0
4,Actinomyces,M0652-1F-1A,0.0


In [30]:
amplicon_melt.head()

,taxa,sampleid,amplicon_abund
0,Acetanaerobacterium,C0047-7E-1A,0.0
1,Acetobacter,C0047-7E-1A,0.0
2,Acholeplasma,C0047-7E-1A,0.0
3,Achromobacter,C0047-7E-1A,0.0
4,Acidaminococcus,C0047-7E-1A,0.0


In [31]:
merged_taxa = pd.merge(amplicon_melt, mgx_melt, on = ["sampleid", "taxa"], how = "outer")

In [32]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [33]:
merged_taxa.sample(10)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
39115,Veillonella,M0770-2E-1A,0.000000,0.0,0.0,0.0
46480,Desulfomicrobium,M0918-3F-1A,0.000000,NaN,NaN,NaN
66769,Propionibacterium,M0644-3E-1A,NaN,0.0,NaN,NaN
51707,Proteus,M0977-2F-1A,0.000000,0.0,0.0,0.0
18329,Fusobacterium,C0789-2F-1A,0.000000,0.0,0.0,0.0
60283,Alteromonas,M1079-1F-1A,0.000000,NaN,NaN,NaN
30068,Parvimonas,M0700-3E-1A,0.000000,0.0,0.0,0.0
5760,Oscillospira,C0494-2E-1A,0.002414,NaN,NaN,NaN
52207,Rubritalea,M0989-1F-1A,0.000000,NaN,NaN,NaN
66016,Tunalikevirus,M0656-2F-1A,NaN,0.0,NaN,NaN


In [34]:
merged_taxa.to_csv('taxa_abundance_comparison.csv')

In [35]:
len(mgx_only)
len(amplicon_only)

59

159

In [36]:
# number of samples where there is one bug picked up by 16S and not mgx

rare_16s = merged_taxa[(merged_taxa.amplicon_abund > 0.0) & (merged_taxa.mgx_abund == 0.0)]
samples_rare_16s = (set(rare_16s["sampleid"]))
num_16s_rare_samples = len(samples_rare_16s)

rare_mgx = merged_taxa[(merged_taxa.mgx_abund > 0.0) & (merged_taxa.amplicon_abund == 0.0)]
samples_rare_mgx = (set(rare_mgx["sampleid"]))
num_mgx_rare_samples = len(samples_rare_mgx)


In [37]:
display(num_16s_rare_samples)
display(num_mgx_rare_samples)

259

258

In [38]:
rare_mgx_taxa = rare_mgx["taxa"]
rare_16s_taxa = rare_16s["taxa"]

In [39]:
len(rare_mgx_taxa)
len(rare_16s_taxa)

923

1456

In [40]:
merged_taxa.head(15)

,taxa,sampleid,amplicon_abund,mgx_abund,abs_diff,tot_diff
0,Acetanaerobacterium,C0047-7E-1A,0.000000,NaN,NaN,NaN
1,Acetobacter,C0047-7E-1A,0.000000,NaN,NaN,NaN
2,Acholeplasma,C0047-7E-1A,0.000000,NaN,NaN,NaN
3,Achromobacter,C0047-7E-1A,0.000000,NaN,NaN,NaN
4,Acidaminococcus,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
5,Acinetobacter,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
6,Actibacter,C0047-7E-1A,0.000000,NaN,NaN,NaN
7,Actinobacillus,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
8,Actinomyces,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000
9,Adlercreutzia,C0047-7E-1A,0.000000,0.000000,0.000000,0.000000


In [41]:
amplicon_avg_abund = merged_taxa.groupby("taxa")["amplicon_abund"].mean()
mgx_avg_abund = merged_taxa.groupby("taxa")["mgx_abund"].mean()
taxa_list = sorted(set(merged_taxa["taxa"]))

In [42]:
mean_taxa_abund = pd.DataFrame(
    (zip(taxa_list, amplicon_avg_abund, mgx_avg_abund)),  
    columns = ['taxa','amp_avg_abund', 'mgx_avg_abund'])

In [43]:
mean_taxa_abund["abs_diff"] = abs(mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"])
mean_taxa_abund["total_diff"] = mean_taxa_abund["amp_avg_abund"] - mean_taxa_abund["mgx_avg_abund"]


In [44]:
mean_taxa_abund.sort_values("abs_diff", axis = 0, ascending = True, 
                 inplace = True, na_position ='last')

In [45]:
mean_taxa_abund.sample(10)

,taxa,amp_avg_abund,mgx_avg_abund,abs_diff,total_diff
191,Mobiluncus,NaN,9.923373e-08,NaN,NaN
195,Muricauda,0.000003,NaN,NaN,NaN
206,Olsenella,0.000061,3.149425e-06,0.000058,0.000058
241,Raoultella,0.000028,3.720575e-05,0.000009,-0.000009
102,Enterorhabdus,0.000008,NaN,NaN,NaN
214,Parabacteroides,0.036607,3.339747e-02,0.003209,0.003209
5,Acinetobacter,0.000032,1.099885e-05,0.000021,0.000021
239,Psychroserpens,0.000008,NaN,NaN,NaN
277,Succiniclasticum,0.000038,NaN,NaN,NaN
142,Helicobacter,NaN,0.000000e+00,NaN,NaN


In [46]:
mean_taxa_abund.to_csv('taxa_difference.csv')

In [47]:
# making giant dataframe of abundances

In [48]:
amp_trans = amplicon.set_index("taxa").transpose()

In [49]:
amp_trans.reset_index(level=0, inplace=True)

In [50]:
amp_trans.rename(columns = {'index':'sampleid'}, inplace = True) 

In [51]:
amp_trans["uid"] = amp_trans["sampleid"].astype(str)+'-amp'

In [52]:
amp_trans["method"] = "amp"

In [53]:
amp_trans.head()

taxa,sampleid,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,Actibacter,Actinobacillus,Actinomyces,...,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts,uid,method
0,C0047-7E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0047-7E-1A-amp,amp
1,C0052-7E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0052-7E-1A-amp,amp
2,C0053-8E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000888,...,0.000634,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0053-8E-1A-amp,amp
3,C0058-4E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.003929,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0058-4E-1A-amp,amp
4,C0059-4E-1A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.001789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C0059-4E-1A-amp,amp


In [79]:
len(amp_trans)

261

In [54]:
mgx_trans = mgx.set_index("taxa").transpose()

In [55]:
mgx_trans.reset_index(level=0, inplace=True)

In [56]:
mgx_trans.rename(columns = {'index':'sampleid'}, inplace = True)

In [57]:
mgx_trans["uid"] = mgx_trans["sampleid"].astype(str)+'-mgx'

In [58]:
mgx_trans["method"] = "mgx"

In [59]:
mgx_trans.head()

taxa,sampleid,Acidaminococcus,Acinetobacter,Actinobacillus,Actinobaculum,Actinomyces,Adlercreutzia,Aeromonas,Aggregatibacter,Akkermansia,...,Tannerella,Treponema,Tunalikevirus,Turicibacter,Ureaplasma,Varibaculum,Veillonella,Weissella,uid,method
0,M0652-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000131,0.0,0.0,0.001095,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0652-1F-1A-mgx,mgx
1,M0647-1F-1A,0.0,0.0,0.0,0.0,0.0,0.001504,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.001681,0.0,0.0,0.001841,0.0,M0647-1F-1A-mgx,mgx
2,M0656-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000907,0.0,0.0,0.019614,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0656-1F-1A-mgx,mgx
3,C0660-1E-1A,0.0,0.0,0.0,0.0,0.0,0.000575,0.0,0.0,0.005417,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000314,0.0,C0660-1E-1A-mgx,mgx
4,M0649-1F-1A,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,M0649-1F-1A-mgx,mgx


In [96]:
concat_df = pd.concat([mgx_trans,amp_trans], sort=True).reset_index(drop = True)

In [97]:
len(concat_df)

522

In [98]:
age = pd.read_csv("~/Documents/thesis/analysis/metadatawide.csv")

In [99]:
age = age[['sample','childAgeMonths']]

In [100]:
age["sample"] = age["sample"].str.replace("_",'-')
age.rename(columns = {'sample':'sampleid'}, inplace = True) 

In [101]:
# make age dictionary
agedict = {str(s): {} for s in age["sampleid"]}
for index, row in age.iterrows():
    age_months = row["childAgeMonths"]
    agedict[row["sampleid"]]= age_months

In [102]:
concat_df["AgeMonths"]= concat_df["sampleid"].map(agedict)

In [103]:
len(concat_df)

522

In [105]:
# remove mothers
concat_df = concat_df[~concat_df.sampleid.str.contains("M")]

In [106]:
len(concat_df)

198

In [68]:
concat_df.sample(15)

,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,Actibacter,Actinobacillus,Actinobaculum,Actinomyces,...,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts,method,sampleid,uid,AgeMonths
12,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,NaN,mgx,C0673-1E-1A,C0673-1E-1A-mgx,46.0
262,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,amp,C0052-7E-1A,C0052-7E-1A-amp,143.0
29,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,NaN,mgx,C0623-1E-1A,C0623-1E-1A-mgx,91.0
221,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,NaN,mgx,C0834-4F-1A,C0834-4F-1A-mgx,13.0
343,0.0,0.0,0.0,0.0,0.000000,0.000032,0.0,0.0,NaN,0.000116,...,0.0,0.0,0.0,0.0,0.0,0.0,amp,C0839-4F-1A,C0839-4F-1A-amp,12.0
334,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,amp,C0788-2F-1A,C0788-2F-1A-amp,109.0
252,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,NaN,mgx,C0828-4F-1A,C0828-4F-1A-mgx,13.0
273,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,amp,C0229-5F-1A,C0229-5F-1A-amp,138.0
322,0.0,0.0,0.0,0.0,0.000126,0.006064,0.0,0.0,NaN,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,amp,C0739-2F-1A,C0739-2F-1A-amp,17.0
247,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,0.0,0.0,0.000000,...,NaN,NaN,0.0,NaN,NaN,NaN,mgx,C0851-4F-1A,C0851-4F-1A-mgx,12.0


In [81]:
len(concat_df)

198

In [69]:
cols_to_order = ['uid', 'sampleid',"method", "AgeMonths"]
new_columns = cols_to_order + (concat_df.columns.drop(cols_to_order).tolist())
concat_df = concat_df[new_columns]

In [70]:
concat_df.sample(15)

,uid,sampleid,method,AgeMonths,Acetanaerobacterium,Acetobacter,Acholeplasma,Achromobacter,Acidaminococcus,Acinetobacter,...,Ureaplasma,Varibaculum,Veillonella,Vibrio,Victivallis,Virgulinella fragilis,Weissella,Winogradskyella,Woeseia,endosymbionts
279,C0388-7F-1A-amp,C0388-7F-1A,amp,81.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,NaN,NaN,0.000334,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
274,C0237-4E-1A-amp,C0237-4E-1A,amp,106.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,NaN,NaN,0.001949,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
325,C0744-4F-1A-amp,C0744-4F-1A,amp,15.0,0.0,0.000000,0.0,0.000000,0.0,0.0,...,NaN,NaN,0.136739,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
179,C0348-3F-1A-mgx,C0348-3F-1A,mgx,124.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000482,NaN,NaN,NaN,0.0,NaN,NaN,NaN
114,C2018-3E-1A-mgx,C2018-3E-1A,mgx,29.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.005355,NaN,NaN,NaN,0.0,NaN,NaN,NaN
115,C0469-2E-1A-mgx,C0469-2E-1A,mgx,128.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000264,NaN,NaN,NaN,0.0,NaN,NaN,NaN
130,C0656-4F-1A-mgx,C0656-4F-1A,mgx,16.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000302,NaN,NaN,NaN,0.0,NaN,NaN,NaN
255,C0461-4F-1A-mgx,C0461-4F-1A,mgx,54.0,NaN,NaN,NaN,NaN,0.0,0.0,...,0.0,0.0,0.000198,NaN,NaN,NaN,0.0,NaN,NaN,NaN
272,C0202-6E-1A-amp,C0202-6E-1A,amp,104.0,0.0,0.000000,0.0,0.044671,0.0,0.0,...,NaN,NaN,0.001852,0.0,0.000000,0.0,0.0,0.0,0.003009,0.0
313,C0703-4F-1A-amp,C0703-4F-1A,amp,15.0,0.0,0.001037,0.0,0.000000,0.0,0.0,...,NaN,NaN,0.007281,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0


In [71]:
concat_df.to_csv('transposed_mgxamp_df.csv')

In [72]:
pwd()

'/Users/danielle/Documents/thesis/analysis'